In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import tensorflow as tf

In [15]:
ranks = ('0', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'j', 'k', 'q', 'z')
rank_model = tf.keras.models.load_model("/home/woojac/proj/031_shuffle/rank_predictor03")

suits = ('0', 'c', 'd', 'h', 's')
suit_model = tf.keras.models.load_model("/home/woojac/proj/031_shuffle/suit_predictor03")

image_size = (224, 224)

In [3]:
# def im_preprocess(im_path):
#     image = tf.keras.preprocessing.image.load_img(im_path, target_size=IMAGE_SIZE)
#     input_arr = tf.keras.preprocessing.image.img_to_array(image)
#     input_arr = np.array([input_arr])

#     normalization_layer = tf.keras.layers.Rescaling(1. / 255)
#     preprocessing_pipe = tf.keras.Sequential([normalization_layer])
#     input_arr = preprocessing_pipe(input_arr)

#     return input_arr

In [4]:
# def frame_preprocess(frame):
#     image = cv2.resize(frame, IMAGE_SIZE)
#     input_arr = image[...,::-1].astype(np.float32)
#     # input_arr = tf.keras.preprocessing.image.img_to_array(image)
#     input_arr = np.array([input_arr])

#     normalization_layer = tf.keras.layers.Rescaling(1. / 255)
#     preprocessing_pipe = tf.keras.Sequential([normalization_layer])
#     input_arr = preprocessing_pipe(input_arr)

#     return input_arr

In [8]:
def prep_frame(f):
    return cv2.resize(f[0:800, 700:1500], (224,224))

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])
def prep_input(inp):
    f = prep_frame(inp)
    arr = np.array([f[...,::-1].astype(np.float32)])
    return preprocessing_model(arr)

In [10]:
def draw_text(img, text, font=cv2.FONT_HERSHEY_PLAIN, pos=(0, 0), font_scale=3, font_thickness=2, text_color=(0, 0, 0), text_color_bg=(255, 255, 255)):
    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)

In [9]:
test_dir = "/home/woojac/proj/031_shuffle/suit_test"
test_pics = os.listdir(test_dir)

for pic in test_pics:
    fullpic = os.path.join(test_dir, pic)
    
    im = im_preprocess(fullpic)
    
    preds = suit_model.predict(im)
    suit_likely = np.max(preds)
    suit_pred = suits[np.argmax(preds)]

    preds = rank_model.predict(im)
    rank_likely = np.max(preds)
    rank_pred = ranks[np.argmax(preds)]
    
    print(rank_pred, rank_likely, suit_pred, suit_likely)
    
    im = cv2.imread(fullpic, 0)
    draw_text(im, rank_pred + suit_pred)
    cv2.imshow("test", im)
    
    if cv2.waitKey() == ord('q'):
        break
    
cv2.destroyAllWindows()

3 4.206448 d 2.8929741
3 4.5590086 c 2.925106
3 4.451889 d 2.8372653
9 4.108052 s 1.8934891
5 3.8893232 s 2.04989
q 4.7998114 s 2.135923
2 4.472737 s 1.9529548
7 4.3941255 h 2.553339
2 4.5558696 s 1.9818089
3 4.3285217 h 2.6408098
5 5.03402 d 2.8254404
3 4.454445 h 2.6068544
2 4.556124 s 1.9976466
9 4.2580466 c 2.8623936
9 4.5715137 c 2.9600174
0 4.6767883 c 2.8023505
a 2.2831507 c 0.84159356


In [25]:
cap = cv2.VideoCapture("/home/woojac/proj/031_shuffle/IMG_0975.MOV")
# cap = cv2.VideoCapture("/home/woojac/proj/031_shuffle/IMG_0982.MOV")

last_card = ""
card_order = []
viz = True

writ = cv2.VideoWriter('savevid.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (1920,1080))

while True:
    ret, frame = cap.read()
    if not ret:
        print("No frame")
        break
            
    inp = prep_input(frame)
    
    preds = suit_model.predict(inp)
    suit_likely = np.max(preds)
    suit_pred = suits[np.argmax(preds)]

    preds = rank_model.predict(inp)
    rank_likely = np.max(preds)
    rank_pred = ranks[np.argmax(preds)]

    if rank_pred!="z" and suit_pred!="0":
        card = rank_pred + suit_pred
        if card == last_card and (len(card_order)==0 or card_order[-1] != card):
            card_order.append(card)
            print(card)
        if card == last_card and viz:
            draw_text(frame, rank_pred + suit_pred, pos=(800, 500))
        last_card = card
        
    
    if viz:
        cv2.imshow('frame', frame)
        writ.write(frame)
        if cv2.waitKey(3) == ord('q'):
            break
        
writ.release()
cv2.destroyAllWindows()

4c
8d
kc
8h
9c
qh
js
3d
2d
kd
9s
9h
0s
6d
ad
2c
2h
3c
7s


In [26]:
print(len(card_order))
print(len(set(card_order)))

19
19


In [8]:
last_card = card_order[-1]

r1 = card_order[:card_order.index(last_card)+1]
r2 = card_order[card_order.index(last_card)+1:]

In [ ]:
## try this 
def get_order(co):
    num_el = len(set(co))
    
    last_card = card_order[-1]

    r1 = card_order[:card_order.index(last_card)+1]
    r2 = card_order[card_order.index(last_card)+1:]

    i = 0
    while i < num_el:
        if r1[i] == r2[i] or r1[i]=="" or r2[i]=="":
            pass
        elif r1[i] == r2[i+1]:
            r1.insert(i,"")
            i = -1
        elif r2[i] == r1[i+1]:
            r2.insert(i,"")
            i = -1
        
        i += 1
        


In [27]:
card_order = ["0d","8c","jc","0h","jh","9s","9d","kh","0s","9h","9d","as","0d","8c","jc","0h","jh","8s","9s","9d","kh","0s","9h"]

In [34]:
num_el = len(set(card_order))

last_card = card_order[-1]

r1 = card_order[:card_order.index(last_card)+1]
r2 = card_order[card_order.index(last_card)+1:]

i = 0
while i < num_el:
    if r1[i] == r2[i] or r1[i]=="  " or r2[i]=="  ":
        pass
    elif i > len(r1):
        r1.append("  ")
        i = -1
    elif i > len(r2):
        r2.append("  ")
        i = -1
    elif r1[i] == r2[i+1]:
        r1.insert(i,"  ")
        i = -1
    elif r2[i] == r1[i+1]:
        r2.insert(i,"  ")
        i = -1
    elif r1[i] == r2[i+2]:
        r1.insert(i,"  ")
        i = -1
    elif r2[i] == r1[i+2]:
        r2.insert(i,"  ")
        i = -1
    else:
        print(r1[i], r1[i+1], r2[i], r2[i+1])
        
    i += 1

[print(i,j) for i,j in zip(r1,r2)]

ord = []
for (e1,e2) in zip(r1, r2):
    if e1 == "  ":
        if card_order.count(e2) > 1: #if there are more elements elsewhere, then this was probably a false id
            continue
        else: #if not, then this was probably a miss
            ord.append(e2)
    elif e2 == "  ":
        if card_order.count(e1) > 1: #if there are more elements elsewhere, then this was probably a false id
            continue
        else: #if not, then this was probably a miss
            ord.append(e1)
    elif e1 == e2:
        ord.append(e1)
    else:
        print("Shouldn't be here!")

ord

   9d
   as
0d 0d
8c 8c
jc jc
0h 0h
jh jh
   8s
9s 9s
9d 9d
kh kh
0s 0s
9h 9h
as
0d
8c
jc
0h
jh
8s
9s
9d
kh
0s
9h


[None, None, None, None, None, None, None, None, None, None, None, None]

In [41]:
def get_shuffle(o1, o2):
    #find first card in o2 that is not in the order of o1
    i = 0
    for e in o2:
        if e==o1[i]:
            i+=1
        else:
            cut_loc = i
    lh = o1[:cut_loc]
    rh = o1[cut_loc:]
    
    o = ""
    il = 0
    ir = 0
    for e in o2:
        if il<len(lh) and e==lh[il]:
            o += "1"
            il += 1
        elif e==rh[ir]:
            o += "2"
            ir += 1
        else:
            ValueError("not feasible shuffle result")
            
    return o

In [42]:
o1 = ["as","0d","8c","jc","0h","jh","8s","9s","9d","kh","0s","9h"]
o2 = ["8s","as","0d","9s","8c","9d","kh","jc","0h","jh","0s","9h"]
get_shuffle(o1,o2)


'211212211122'